# Import

In [1]:
import pandas as pd
import json

from curation_tools.curation_tools import (
    CuratedDataset,
    ObsSchema,
    VarSchema,
    Experiment,
    download_file,
    upload_parquet_to_bq
)

import logging
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)s %(name)s: %(message)s",
    handlers=[
        logging.FileHandler("curation.log"),
        logging.StreamHandler(),  # keep console output too
    ],
    force=True,
)

/Users/zakirov/Documents/GitHub/PerturbationCatalogue/.venv/lib/python3.12/site-packages/pandera/_pandas_deprecated.py:157: FutureWarning: Importing pandas-specific classes and functions from the
top-level pandera module will be **removed in a future version of pandera**.
If you're using pandera to validate pandas objects, we highly recommend updating
your import:

```
# old import
import pandera as pa

# new import
import pandera.pandas as pa
```

If you're using pandera to validate objects from other compatible libraries
like pyspark or polars, see the supported libraries section of the documentation
for more information on how to import pandera:

https://pandera.readthedocs.io/en/stable/supported_libraries.html

To disable this warning, set the environment variable:

```
export DISABLE_PANDERA_IMPORT_WARNING=True
```

  warnings.warn(_future_warning, FutureWarning)


# Download the dataset

In [2]:
noncurated_path = "../non_curated/h5ad/datlinger_2017.h5ad"
download_file(
    url="https://zenodo.org/records/13350497/files/DatlingerBock2017.h5ad",
    dest_path=noncurated_path
)

File ../non_curated/h5ad/datlinger_2017.h5ad already exists. Skipping download.


# Initialise the dataset object

In [3]:
cur_data = CuratedDataset(
    obs_schema=ObsSchema,
    var_schema=VarSchema,
    exp_metadata_schema=Experiment,
    noncurated_path=noncurated_path
)

cur_data.load_data()

2025-12-12 11:16:40,089 DEBUG h5py._conv: Creating converter from 3 to 5


Loading data from ../non_curated/h5ad/datlinger_2017.h5ad


In [4]:
cur_data.show_var()

Variable data:
DataFrame shape: (36722, 2)
--------------------------------------------------
              ncounts  ncells
gene_symbol                  
A1BG             60.0      57
A1BG-AS1        324.0     306
A1CF              1.0       1
A2M               6.0       6
A2M-AS1           1.0       1
...               ...     ...
hsa-mir-1587      7.0       7
hsa-mir-3149      0.0       0
hsa-mir-4259      1.0       1
snosnR66          1.0       1
yR211F11.2        3.0       3

[36722 rows x 2 columns]
--------------------------------------------------


# OBS slot curation

### Show unique perturbations

In [5]:
cur_data.show_unique(slot = 'obs', column = 'perturbation')

Unique values in adata.obs.perturbation: 97
--------------------------------------------------
{'Essential_library_DHODH_1',
 'Essential_library_DHODH_2',
 'Essential_library_DHODH_3',
 'Essential_library_MVD_1',
 'Essential_library_MVD_2',
 'Essential_library_MVD_3',
 'Essential_library_TUBB_1',
 'Essential_library_TUBB_2',
 'Essential_library_TUBB_3',
 'Tcrlibrary_BACH2_1',
 'Tcrlibrary_BACH2_2',
 'Tcrlibrary_BACH2_3',
 'Tcrlibrary_DOK2_1',
 'Tcrlibrary_DOK2_2',
 'Tcrlibrary_DOK2_3',
 'Tcrlibrary_EGR1_1',
 'Tcrlibrary_EGR1_2',
 'Tcrlibrary_EGR1_3',
 'Tcrlibrary_EGR2_1',
 'Tcrlibrary_EGR2_2',
 'Tcrlibrary_EGR2_3',
 'Tcrlibrary_EGR3_1',
 'Tcrlibrary_EGR3_2',
 'Tcrlibrary_EGR3_3',
 'Tcrlibrary_EGR4_1',
 'Tcrlibrary_EGR4_2',
 'Tcrlibrary_EGR4_3',
 'Tcrlibrary_ETS1_1',
 'Tcrlibrary_ETS1_2',
 'Tcrlibrary_ETS1_3',
 'Tcrlibrary_FOS_1',
 'Tcrlibrary_FOS_2',
 'Tcrlibrary_FOS_3',
 'Tcrlibrary_GATA3_1',
 'Tcrlibrary_GATA3_2',
 'Tcrlibrary_GATA3_3',
 'Tcrlibrary_JUNB_1',
 'Tcrlibrary_JUNB_2',
 'T

### Rename `perturbation` to `perturbation_name`

In [6]:
cur_data.rename_columns(slot = 'obs', name_dict = {'perturbation': 'perturbation_name'})

Renamed columns in adata.obs: {'perturbation': 'perturbation_name'}


### Add guide RNA information

Data from perturbseq has non-uniquely identifiable control cells labelled as "control". This prevents unique assingnement of guide RNA information to these cells.

However, the original count matrix and supplementary data file contains this information, so we have to add it manually.

Original data: https://ftp.ncbi.nlm.nih.gov/geo/series/GSE92nnn/GSE92872/suppl/GSE92872%5FCROP%2Dseq%5FJurkat%5FTCR.digital%5Fexpression.csv.gz

Supplementary table S2: https://static-content.springer.com/esm/art%3A10.1038%2Fnmeth.4177/MediaObjects/41592_2017_BFnmeth4177_MOESM268_ESM.xlsx

In [7]:
# read data from supplementary files
cell_ids_df = pd.read_csv("../supplementary/datlinger_2017_cell_ids.csv")
guide_ids_df = pd.read_csv("../supplementary/datlinger_2017_guide_ids.csv")
guide_ids_df = guide_ids_df[["gRNA_ID", "Sequence"]]

# merge the dataframes to get the guide sequences for each cell
guides_df = cell_ids_df.merge(
    guide_ids_df, left_on="grna", right_on="gRNA_ID", how="left", indicator=True
)
# convert the dataframe to a dictionary with cell barcodes as keys and guide sequences as values
guides_dict_vecid = guides_df.set_index("cell")["Sequence"].to_dict()

guides_dict_vecid

{'TACTTGACCCCN': 'GAGAACGTGATAAGACTCGG',
 'TTACAGCTGAAC': 'GAACTGTGAGCTCGTCGGCG',
 'CTAAGGCCCTTA': 'CAGCACAGCGGATGACCTCG',
 'CTTGACGCAGGT': 'CGAGGGACCAGCCAAGATCG',
 'TAACCCGTACGC': 'AAGGTCCGCTCTCGGACGGG',
 'ATCTAGATACNN': 'GAATGCTGAGTACGGTCTGT',
 'CTATCGTTCTTN': 'CCATCCCATGGTGGACTACC',
 'GTATTGCGAGCN': 'TACGGCGATGAGGCGCTGAG',
 'GTACTGTGTTAN': 'CAGTACTTTTACCCCCGCGG',
 'CGTCTTTCANNN': 'TAGGCTGTTCCACGATCACC',
 'CAGTTTATTCAC': 'CTACTACGGAAACTCGGTCA',
 'CTTGTTGCCTGT': 'TACGTGCCCGAGTACAGCTC',
 'GCATCAGTAATN': 'GGTGATCCTAGTCGACTGGC',
 'CCGCCAGCTCTC': 'CCCGTATGAGCTTCGGATTG',
 'AATCTTGTAGGG': 'CAGCACAGCGGATGACCTCG',
 'AAGCCACCTGAN': 'CAAGTGCGAGGGGCGCTCCG',
 'CAATAAGGTGNN': 'TCCTCCGGCCGACGCCTTCA',
 'CCAGGTTACTTG': 'CAGAAACCCATGTTCGGGAC',
 'GATCAGCCCGAG': 'GAACTGTGAGCTCGTCGGCG',
 'TGATGGCGTGAA': 'GCCCCACGACAACCGCACCA',
 'GTCTAATGTCNN': 'GCGCATCAATATGCCCGCAC',
 'TAGTATTTTCCN': 'CCCGTATGAGCTTCGGATTG',
 'CGGCTGCTCATN': 'GCTCAATGATCTCCACATAG',
 'CTCCGCCCCTGN': 'TCGGTGCGCACCAGCTCGCC',
 'GTTCTACGATNN':

In [8]:
# create a cell barcode column for mapping
cur_data.create_columns(
    slot="obs",
    col_dict={
        "cell_bc": cur_data.adata.obs.index
    },
)

# map the guide sequences to the cell barcodes
cur_data.map_values_from_column(
    ref_col="cell_bc", target_col="guide_sequence", map_dict=guides_dict_vecid
)


Column cell_bc added to adata.obs
Column guide_sequence created in adata.obs
Mapped value TACTTGACCCCN in column cell_bc to GAGAACGTGATAAGACTCGG in column guide_sequence of adata.obs
Mapped value TTACAGCTGAAC in column cell_bc to GAACTGTGAGCTCGTCGGCG in column guide_sequence of adata.obs
Mapped value CTAAGGCCCTTA in column cell_bc to CAGCACAGCGGATGACCTCG in column guide_sequence of adata.obs
Mapped value CTTGACGCAGGT in column cell_bc to CGAGGGACCAGCCAAGATCG in column guide_sequence of adata.obs
Mapped value TAACCCGTACGC in column cell_bc to AAGGTCCGCTCTCGGACGGG in column guide_sequence of adata.obs
Mapped value ATCTAGATACNN in column cell_bc to GAATGCTGAGTACGGTCTGT in column guide_sequence of adata.obs
Mapped value CTATCGTTCTTN in column cell_bc to CCATCCCATGGTGGACTACC in column guide_sequence of adata.obs
Mapped value GTATTGCGAGCN in column cell_bc to TACGGCGATGAGGCGCTGAG in column guide_sequence of adata.obs
Mapped value GTACTGTGTTAN in column cell_bc to CAGTACTTTTACCCCCGCGG in colu

### Extract perturbation symbols

#### Add `perturbation_target_symbol` column based on the `perturbation_name`

In [9]:
cur_data.create_columns(
    slot="obs", col_dict={"perturbed_target_symbol_input": cur_data.adata.obs["perturbation_name"]},
    overwrite=True
)
cur_data.show_unique(slot = 'obs', column = 'perturbed_target_symbol_input')

Column perturbed_target_symbol_input added to adata.obs
Unique values in adata.obs.perturbed_target_symbol_input: 97
--------------------------------------------------
{'Essential_library_DHODH_1',
 'Essential_library_DHODH_2',
 'Essential_library_DHODH_3',
 'Essential_library_MVD_1',
 'Essential_library_MVD_2',
 'Essential_library_MVD_3',
 'Essential_library_TUBB_1',
 'Essential_library_TUBB_2',
 'Essential_library_TUBB_3',
 'Tcrlibrary_BACH2_1',
 'Tcrlibrary_BACH2_2',
 'Tcrlibrary_BACH2_3',
 'Tcrlibrary_DOK2_1',
 'Tcrlibrary_DOK2_2',
 'Tcrlibrary_DOK2_3',
 'Tcrlibrary_EGR1_1',
 'Tcrlibrary_EGR1_2',
 'Tcrlibrary_EGR1_3',
 'Tcrlibrary_EGR2_1',
 'Tcrlibrary_EGR2_2',
 'Tcrlibrary_EGR2_3',
 'Tcrlibrary_EGR3_1',
 'Tcrlibrary_EGR3_2',
 'Tcrlibrary_EGR3_3',
 'Tcrlibrary_EGR4_1',
 'Tcrlibrary_EGR4_2',
 'Tcrlibrary_EGR4_3',
 'Tcrlibrary_ETS1_1',
 'Tcrlibrary_ETS1_2',
 'Tcrlibrary_ETS1_3',
 'Tcrlibrary_FOS_1',
 'Tcrlibrary_FOS_2',
 'Tcrlibrary_FOS_3',
 'Tcrlibrary_GATA3_1',
 'Tcrlibrary_GATA3_2

#### Clean up `perturbation_target_symbol` column

In [10]:
cur_data.replace_entries(
    slot="obs",
    column="perturbed_target_symbol_input",
    map_dict={
        "Essential_library_": "",
        "Tcrlibrary_": "",
        "_(1|2|3)": "",
        "control": "control_nontargeting",
    },
)

cur_data.show_unique(slot="obs", column="perturbed_target_symbol_input")

Replaced 'Essential_library_' with '' in column perturbed_target_symbol_input of adata.obs
Replaced 'Tcrlibrary_' with '' in column perturbed_target_symbol_input of adata.obs
Replaced '_(1|2|3)' with '' in column perturbed_target_symbol_input of adata.obs
Replaced 'control' with 'control_nontargeting' in column perturbed_target_symbol_input of adata.obs
Unique values in adata.obs.perturbed_target_symbol_input: 33
--------------------------------------------------
{'BACH2',
 'DHODH',
 'DOK2',
 'EGR1',
 'EGR2',
 'EGR3',
 'EGR4',
 'ETS1',
 'FOS',
 'GATA3',
 'JUN',
 'JUNB',
 'JUND',
 'LAT',
 'LCK',
 'MVD',
 'NFAT5',
 'NFATC1',
 'NFATC2',
 'NFATC3',
 'NFKB1',
 'NFKB2',
 'NR4A1',
 'PTPN11',
 'PTPN6',
 'REL',
 'RELA',
 'RELB',
 'RUNX1',
 'RUNX2',
 'TUBB',
 'ZAP70',
 'control_nontargeting'}
--------------------------------------------------


/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:609: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  if df[column].str.upper().str.contains(old_val.upper()).any():


### Standardise perturbation targets

In [11]:
cur_data.standardize_genes(
    slot='obs',
    input_column='perturbed_target_symbol_input',
    input_column_type='gene_symbol',
    multiple_entries=False
)

Mapping gene symbols: 100%|███████████████████████████████████████| 33/33 [00:00<00:00, 9192.54it/s]


--------------------------------------------------
Successfully mapped 32 out of 33 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['control_nontargeting']
--------------------------------------------------


### Add `perturbed_target_number` column

In [12]:
cur_data.count_entries(
    slot='obs',
    input_column='perturbed_target_symbol',
    count_column_name='perturbed_target_number',
    sep='|'
)

Counted entries in column perturbed_target_symbol of adata.obs and stored in perturbed_target_number


### Encode chromosomes as integers

In [13]:
cur_data.chromosome_encoding()

Chromosome encoding applied to perturbed_target_chromosome in adata.obs and stored as 'perturbed_target_chromosome_encoding'.


In [14]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_chromosome_encoding'])

Observation data:
DataFrame shape: (5905, 2)
--------------------------------------------------
               perturbation_name  perturbed_target_chromosome_encoding
index                                                                 
TACTTGACCCCN             control                                     0
TTACAGCTGAAC   Tcrlibrary_JUND_2                                    19
CTAAGGCCCTTA  Tcrlibrary_BACH2_3                                     6
CTTGACGCAGGT  Tcrlibrary_NFKB2_3                                    10
TAACCCGTACGC    Tcrlibrary_JUN_1                                     1
...                          ...                                   ...
GTGTGTCGGGGA             control                                     0
TTTAGTATTCCA   Tcrlibrary_EGR1_3                                     5
GGCGCCTAATCG             control                                     0
GCTGAGCGTTTN  Tcrlibrary_NR4A1_1                                    12
ACTCGCAGAGAT    Tcrlibrary_REL_3                    

In [15]:
cur_data.adata.obs

,perturbation_type_2,cell_line,perturbation_type,target,percent_mito,tissue_type,replicate,organism,perturbed_target_symbol_input,nperts,...,percent_ribo,perturbation_name,cell_bc,perturbed_target_ensg,perturbed_target_symbol,perturbed_target_biotype,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_number,perturbed_target_chromosome_encoding
index,,,,,,,,,,,,,,,,,,,,,
TACTTGACCCCN,TCR stimulation,Jurkat cells,CRISPR,NaN,0.287488,cell_line,1,human,control_nontargeting,1,...,25.091997,control,TACTTGACCCCN,None,control_nontargeting,None,None,None,1,0
TTACAGCTGAAC,TCR stimulation,Jurkat cells,CRISPR,JUND,5.065666,cell_line,1,human,JUND,3,...,15.384615,Tcrlibrary_JUND_2,TTACAGCTGAAC,ENSG00000130522,JUND,protein_coding,chr19:18279694-18281622;-1,19,1,19
CTAAGGCCCTTA,TCR stimulation,Jurkat cells,CRISPR,BACH2,4.350498,cell_line,1,human,BACH2,3,...,20.412930,Tcrlibrary_BACH2_3,CTAAGGCCCTTA,ENSG00000112182,BACH2,protein_coding,chr6:89926528-90296923;-1,6,1,6
CTTGACGCAGGT,TCR stimulation,Jurkat cells,CRISPR,NFKB2,9.034180,cell_line,1,human,NFKB2,3,...,10.466600,Tcrlibrary_NFKB2_3,CTTGACGCAGGT,ENSG00000077150,NFKB2,protein_coding,chr10:102394110-102402529;1,10,1,10
TAACCCGTACGC,TCR stimulation,Jurkat cells,CRISPR,JUN,12.140106,cell_line,1,human,JUN,3,...,3.594352,Tcrlibrary_JUN_1,TAACCCGTACGC,ENSG00000177606,JUN,protein_coding,chr1:58776845-58784048;-1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTGTGTCGGGGA,TCR stimulation,Jurkat cells,CRISPR,NaN,4.486115,cell_line,5,human,control_nontargeting,1,...,16.496559,control,GTGTGTCGGGGA,None,control_nontargeting,None,None,None,1,0
TTTAGTATTCCA,TCR stimulation,Jurkat cells,CRISPR,EGR1,1.834644,cell_line,5,human,EGR1,3,...,14.939242,Tcrlibrary_EGR1_3,TTTAGTATTCCA,ENSG00000120738,EGR1,protein_coding,chr5:138465479-138469303;1,5,1,5
GGCGCCTAATCG,TCR stimulation,Jurkat cells,CRISPR,NaN,2.213439,cell_line,5,human,control_nontargeting,1,...,17.891964,control,GGCGCCTAATCG,None,control_nontargeting,None,None,None,1,0


### Add metadata

In [16]:
cur_data.create_columns(
    slot="obs",
    col_dict={
        "dataset_id": cur_data.dataset_id,
        "sample_id": range(1, cur_data.adata.obs.shape[0] + 1),
        # treatment
        "treatment_label": cur_data.adata.obs['perturbation_2'],
        "treatment_id": cur_data.adata.obs['perturbation_2'],
        # perturbation type
        "perturbation_type_label": "CRISPRn", 
        "perturbation_type_id": None,
        "data_modality": "CRISPR screen",
        "significant": None,
        "significance_criteria": None,
        "score_interpretation": None,
        # model system
        "model_system_label": "cell_line",
        "model_system_id": None,
        # replicated
        "technical_replicate": None,
        "biological_replicate": None,

        'tissue': 'blood',
        "timepoint": "P0DT0H0M0S",
        
        "species": "Homo sapiens",
        "sex_label": "male", 
        "sex_id": None,
        "developmental_stage_label": "adolescent", 
        "developmental_stage_id": None,

        "study_title": "Pooled CRISPR screening with single-cell transcriptome readout",
        "study_uri": "https://doi.org/10.1038/nmeth.4177",
        "study_year": 2017,
        "first_author": "Paul Datlinger",
        "last_author": "Christoph Bock",
        
        "experiment_title": "Transcriptomics measurements of 5905 Jurkat cells induced with anti-CD3 and anti-CD28 antibodies",
        "experiment_summary": """
            Jurkat cells were transduced with a gRNA library targeting high-level
            regulators of T cell receptor signaling and a set of transcription factors. After 10
            days of antibiotic selection and expansion, cells were stimulated with anti-CD3 and
            anti-CD28 antibodies or left untreated. Both conditions were analyzed using CROP-seq,
            measuring TCR activation for each gene knockout. The dataset comprises 5,905 high-quality
            single-cell transcriptomes with uniquely assigned gRNAs.
            """,
            
        "number_of_perturbed_targets": len(set(cur_data.adata.obs['perturbed_target_coord'])),
        "number_of_perturbed_samples": cur_data.adata.obs.shape[0],
        
        "library_generation_type_id": "EFO:0022868",
        "library_generation_type_label": "endogenous",

        "library_generation_method_id": "EFO:0022876",
        "library_generation_method_label": "SpCas9",

        "enzyme_delivery_method_id": None,
        "enzyme_delivery_method_label": "lentivirus transduction",

        "library_delivery_method_id": None,
        "library_delivery_method_label": "lentivirus transduction",

        "enzyme_integration_state_id": None,
        "enzyme_integration_state_label": "random locus integration",

        "library_integration_state_id": None,
        "library_integration_state_label": "random locus integration",

        "enzyme_expression_control_id": None,
        "enzyme_expression_control_label": "constitutive transgene expression",

        "library_expression_control_id": None,
        "library_expression_control_label": "constitutive transgene expression",

        "library_name": "custom",
        "library_uri": None,

        "library_format_id": None,
        "library_format_label": "pooled",

        "library_scope_id": None,
        "library_scope_label": "focused",

        "library_perturbation_type_id": None,
        "library_perturbation_type_label": "knockout",
        
        "library_manufacturer": "Bock",
        "library_lentiviral_generation": "3",
        "library_grnas_per_target": "3",
        "library_total_grnas": "116",
        "library_total_variants": None,
        
        "readout_dimensionality_id": None,
        "readout_dimensionality_label": "high-dimensional assay",

        "readout_type_id": None,
        "readout_type_label": "transcriptomic",

        "readout_technology_id": None,
        "readout_technology_label": "single-cell rna-seq",

        "method_name_id": None,
        "method_name_label": "Perturb-seq",

        "method_uri": None,

        "sequencing_library_kit_id": None,
        "sequencing_library_kit_label": "Nextera XT DNA Library Preparation Kit",

        "sequencing_platform_id": None,
        "sequencing_platform_label": "Illumina HiSeq 4000",

        "sequencing_strategy_id": None,
        "sequencing_strategy_label": "barcode sequencing",

        "software_counts_id": None,
        "software_counts_label": "Drop-seq Tools",

        "software_analysis_id": None,
        "software_analysis_label": "custom",

        "reference_genome_id": None,
        "reference_genome_label": "GRCh38",
        
        "license_label": "CC BY 4.0",
        "license_id": "SWO:1000065",

        "associated_datasets": json.dumps([
            {
                "dataset_accession": "GSE92872",
                "dataset_uri": "https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE92872",
                "dataset_description": "Digital expression matrix",
                "dataset_file_name": "GSE92872_CROP-seq_Jurkat_TCR.digital_expression.csv.gz",
            },
            {
                "dataset_accession": "DatlingerBock2017.h5ad",
                "dataset_uri": "https://zenodo.org/records/13350497/files/DatlingerBock2017.h5ad",
                "dataset_description": "Processed .h5ad file",
                "dataset_file_name": "DatlingerBock2017.h5ad",
            }
        ])
    }
)

Column dataset_id added to adata.obs
Column sample_id added to adata.obs
Column treatment_label added to adata.obs
Column treatment_id added to adata.obs
Column perturbation_type_label added to adata.obs
Column perturbation_type_id added to adata.obs
Column data_modality added to adata.obs
Column significant added to adata.obs
Column significance_criteria added to adata.obs
Column score_interpretation added to adata.obs
Column model_system_label added to adata.obs
Column model_system_id added to adata.obs
Column technical_replicate added to adata.obs
Column biological_replicate added to adata.obs
Column tissue added to adata.obs
Column timepoint added to adata.obs
Column species added to adata.obs
Column sex_label added to adata.obs
Column sex_id added to adata.obs
Column developmental_stage_label added to adata.obs
Column developmental_stage_id added to adata.obs
Column study_title added to adata.obs
Column study_uri added to adata.obs
Column study_year added to adata.obs
Column first

In [17]:
cur_data.adata.obs

,perturbation_type_2,cell_line,perturbation_type,target,percent_mito,tissue_type,replicate,organism,perturbed_target_symbol_input,nperts,...,sequencing_strategy_label,software_counts_id,software_counts_label,software_analysis_id,software_analysis_label,reference_genome_id,reference_genome_label,license_label,license_id,associated_datasets
index,,,,,,,,,,,,,,,,,,,,,
TACTTGACCCCN,TCR stimulation,Jurkat cells,CRISPR,NaN,0.287488,cell_line,1,human,control_nontargeting,1,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
TTACAGCTGAAC,TCR stimulation,Jurkat cells,CRISPR,JUND,5.065666,cell_line,1,human,JUND,3,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
CTAAGGCCCTTA,TCR stimulation,Jurkat cells,CRISPR,BACH2,4.350498,cell_line,1,human,BACH2,3,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
CTTGACGCAGGT,TCR stimulation,Jurkat cells,CRISPR,NFKB2,9.034180,cell_line,1,human,NFKB2,3,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
TAACCCGTACGC,TCR stimulation,Jurkat cells,CRISPR,JUN,12.140106,cell_line,1,human,JUN,3,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTGTGTCGGGGA,TCR stimulation,Jurkat cells,CRISPR,NaN,4.486115,cell_line,5,human,control_nontargeting,1,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
TTTAGTATTCCA,TCR stimulation,Jurkat cells,CRISPR,EGR1,1.834644,cell_line,5,human,EGR1,3,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."
GGCGCCTAATCG,TCR stimulation,Jurkat cells,CRISPR,NaN,2.213439,cell_line,5,human,control_nontargeting,1,...,barcode sequencing,None,Drop-seq Tools,None,custom,None,GRCh38,CC BY 4.0,SWO:1000065,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur..."


### Curate treatment information

In [18]:
cur_data.replace_entries(
    slot='obs',
    column="treatment_label",
    map_dict={
        'unstimulated': 'Untreated Control',
        'stimulated': 'anti-CD3 antibody|anti-CD28 antibody'
    }
)

cur_data.replace_entries(
    slot='obs',
    column="treatment_id",
    map_dict={
        'unstimulated': 'NCIT:C184729',
        'stimulated': 'EFO:0003317|EFO:0003304'
    }
)

Replaced 'unstimulated' with 'Untreated Control' in column treatment_label of adata.obs
Replaced 'stimulated' with 'anti-CD3 antibody|anti-CD28 antibody' in column treatment_label of adata.obs
Replaced 'unstimulated' with 'NCIT:C184729' in column treatment_id of adata.obs
Replaced 'stimulated' with 'EFO:0003317|EFO:0003304' in column treatment_id of adata.obs


### Curate tissue information

In [19]:

cur_data.standardize_ontology(
    input_column='tissue',
    column_type='term_name',
    ontology_type='tissue',
    overwrite=True
)

Mapped 1 tissue ontology terms from `tissue` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
  input_column input_column_lower name_lower     ontology_id
0        blood              blood      blood  UBERON:0000178
--------------------------------------------------


/Users/zakirov/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell type information

In [20]:
cur_data.standardize_ontology(
    input_column='celltype',
    column_type='term_name',
    ontology_type='cell_type'
)

Mapped 1 cell_type ontology terms from `celltype` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
  input_column input_column_lower name_lower ontology_id
0      T cells            t cells    t cells  CL:0000084
--------------------------------------------------


/Users/zakirov/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate cell line information

In [21]:
cur_data.standardize_ontology(
    input_column='cell_line',
    column_type='term_name',
    ontology_type='cell_line'
)

Mapped 1 cell_line ontology terms from `cell_line` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
   input_column input_column_lower    name_lower  ontology_id
0  Jurkat cells       jurkat cells  jurkat cells  CLO:0007043
--------------------------------------------------


/Users/zakirov/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Curate disease information

In [22]:
cur_data.standardize_ontology(
    input_column='disease',
    column_type='term_name',
    ontology_type='disease'
)

Mapped 1 disease ontology terms from `disease` column to ontology terms
DataFrame shape: (1, 4)
--------------------------------------------------
            input_column     input_column_lower             name_lower  \
0  acute T cell leukemia  acute t cell leukemia  acute t cell leukemia   

     ontology_id  
0  MONDO:0004963  
--------------------------------------------------


/Users/zakirov/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/functools.py:909: ImplicitModificationWarning: Transforming to str index.
  return dispatch(args[0].__class__)(*args, **kw)


### Match schema column order

In [23]:
cur_data.match_schema_columns(slot='obs')

Matched columns of adata.obs to the obs_schema.


### Validate obs metadata

In [24]:
cur_data.validate_data(slot='obs')

2025-12-12 11:16:51,878 DEBUG curation_tools.curation_tools: Applying dtype casting on adata.obs for columns: ['dataset_id', 'sample_id', 'data_modality', 'significant', 'significance_criteria', 'perturbation_name', 'perturbed_target_coord', 'perturbed_target_chromosome', 'perturbed_target_chromosome_encoding', 'perturbed_target_number', 'perturbed_target_ensg', 'perturbed_target_symbol', 'perturbed_target_biotype', 'guide_sequence', 'perturbation_type_label', 'perturbation_type_id', 'timepoint', 'treatment_label', 'treatment_id', 'technical_replicate', 'biological_replicate', 'model_system_label', 'model_system_id', 'species', 'tissue_label', 'tissue_id', 'cell_type_label', 'cell_type_id', 'cell_line_label', 'cell_line_id', 'sex_label', 'sex_id', 'developmental_stage_label', 'developmental_stage_id', 'disease_label', 'disease_id', 'study_title', 'study_uri', 'study_year', 'first_author', 'last_author', 'experiment_title', 'experiment_summary', 'number_of_perturbed_targets', 'number_of

,dataset_id,sample_id,data_modality,significant,significance_criteria,perturbation_name,perturbed_target_coord,perturbed_target_chromosome,perturbed_target_chromosome_encoding,perturbed_target_number,...,software_counts_id,software_counts_label,software_analysis_id,software_analysis_label,score_interpretation,reference_genome_id,reference_genome_label,associated_datasets,license_label,license_id
0,datlinger_2017,1,CRISPR screen,<NA>,<NA>,control,<NA>,<NA>,0,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
1,datlinger_2017,2,CRISPR screen,<NA>,<NA>,Tcrlibrary_JUND_2,chr19:18279694-18281622;-1,19,19,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
2,datlinger_2017,3,CRISPR screen,<NA>,<NA>,Tcrlibrary_BACH2_3,chr6:89926528-90296923;-1,6,6,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
3,datlinger_2017,4,CRISPR screen,<NA>,<NA>,Tcrlibrary_NFKB2_3,chr10:102394110-102402529;1,10,10,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
4,datlinger_2017,5,CRISPR screen,<NA>,<NA>,Tcrlibrary_JUN_1,chr1:58776845-58784048;-1,1,1,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5900,datlinger_2017,5901,CRISPR screen,<NA>,<NA>,control,<NA>,<NA>,0,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
5901,datlinger_2017,5902,CRISPR screen,<NA>,<NA>,Tcrlibrary_EGR1_3,chr5:138465479-138469303;1,5,5,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
5902,datlinger_2017,5903,CRISPR screen,<NA>,<NA>,control,<NA>,<NA>,0,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065
5903,datlinger_2017,5904,CRISPR screen,<NA>,<NA>,Tcrlibrary_NR4A1_1,chr12:52022832-52059509;1,12,12,1,...,<NA>,Drop-seq Tools,<NA>,custom,<NA>,<NA>,GRCh38,"[{""dataset_accession"": ""GSE92872"", ""dataset_ur...",CC BY 4.0,SWO:1000065


In [25]:
cur_data.show_obs(['perturbation_name', 'perturbed_target_symbol', 'perturbed_target_ensg', 'perturbed_target_coord'])

Observation data:
DataFrame shape: (5905, 4)
--------------------------------------------------
       perturbation_name perturbed_target_symbol perturbed_target_ensg  \
0                control    control_nontargeting                  <NA>   
1      Tcrlibrary_JUND_2                    JUND       ENSG00000130522   
2     Tcrlibrary_BACH2_3                   BACH2       ENSG00000112182   
3     Tcrlibrary_NFKB2_3                   NFKB2       ENSG00000077150   
4       Tcrlibrary_JUN_1                     JUN       ENSG00000177606   
...                  ...                     ...                   ...   
5900             control    control_nontargeting                  <NA>   
5901   Tcrlibrary_EGR1_3                    EGR1       ENSG00000120738   
5902             control    control_nontargeting                  <NA>   
5903  Tcrlibrary_NR4A1_1                   NR4A1       ENSG00000123358   
5904    Tcrlibrary_REL_3                     REL       ENSG00000162924   

           pert

# VAR slot curation

### Standardise genes

In [29]:
cur_data.show_var()

Variable data:
DataFrame shape: (36722, 5)
--------------------------------------------------
              ncells gene_symbol_input  ncounts  ensembl_gene_id gene_symbol
index                                                                       
A1BG              57              A1BG     60.0  ENSG00000121410        A1BG
A1BG-AS1         306          A1BG-AS1    324.0  ENSG00000268895    A1BG-AS1
A1CF               1              A1CF      1.0  ENSG00000148584        A1CF
A2M                6               A2M      6.0  ENSG00000175899         A2M
A2M-AS1            1           A2M-AS1      1.0  ENSG00000245105     A2M-AS1
...              ...               ...      ...              ...         ...
hsa-mir-1587       7      hsa-mir-1587      7.0  ENSG00000263972     MIR1587
hsa-mir-3149       0      hsa-mir-3149      0.0  ENSG00000266712     MIR3149
hsa-mir-4259       1      hsa-mir-4259      1.0  ENSG00000266458     MIR4259
snosnR66           1          snosnR66      1.0            

In [27]:
cur_data.create_columns(
    slot = 'var', 
    col_dict={'gene_symbol_input': cur_data.adata.var.index},
    overwrite=True
)

Column gene_symbol_input added to adata.var


In [28]:
cur_data.standardize_genes(
    slot="var",
    input_column="gene_symbol_input",
    input_column_type="gene_symbol",
    remove_version=True,
    multiple_entries=False
)

Removed version numbers from gene_symbol_input


Mapping gene symbols: 100%|████████████████████████████████| 32750/32750 [00:00<00:00, 44277.81it/s]


--------------------------------------------------
Successfully mapped 25582 out of 32750 gene symbols.
--------------------------------------------------
Couldn't map gene symbols: ['AB019438', 'AB019440', 'AB019441', 'ABBA01014710', 'ABBA01017803', 'ABBA01031658', 'ABBA01031664', 'ABC12-49244600F4', 'ABC14-1080714F14', 'AC000362', 'AC002069', 'AC002331', 'AC002454', 'AC002523', 'AC002539', 'AC003009', 'AC003090', 'AC003092', 'AC004009', 'AC004014', 'AC004053', 'AC004158', 'AC004166', 'AC004386', 'AC004535', 'AC004538', 'AC004870', 'AC004878', 'AC004941', 'AC004987', 'AC005019', 'AC005062', 'AC005094', 'AC005150', 'AC005152', 'AC005176', 'AC005304', 'AC005307', 'AC005394', 'AC005487', 'AC005523', 'AC005562', 'AC005616', 'AC005754', 'AC005758', 'AC005772', 'AC005863', 'AC006004', 'AC006041', 'AC006129', 'AC006296', 'AC006355', 'AC006499', 'AC006840', 'AC006988', 'AC006989', 'AC006995', 'AC007064', 'AC007106', 'AC007126', 'AC007131', 'AC007179', 'AC007238', 'AC007389', 'AC007463', 'AC00

### Validate var metadata

In [30]:
cur_data.validate_data(slot='var')

2025-12-12 11:19:35,144 INFO curation_tools.curation_tools: adata.var is valid according to the var_schema.
2025-12-12 11:19:35,206 DEBUG curation_tools.curation_tools: Validated adata.var preview (shape=(36722, 2)):
          ensembl_gene_id gene_symbol
index                                
A1BG      ENSG00000121410        A1BG
A1BG-AS1  ENSG00000268895    A1BG-AS1
A1CF      ENSG00000148584        A1CF
A2M       ENSG00000175899         A2M
A2M-AS1   ENSG00000245105     A2M-AS1


,ensembl_gene_id,gene_symbol
index,,
A1BG,ENSG00000121410,A1BG
A1BG-AS1,ENSG00000268895,A1BG-AS1
A1CF,ENSG00000148584,A1CF
A2M,ENSG00000175899,A2M
A2M-AS1,ENSG00000245105,A2M-AS1
...,...,...
hsa-mir-1587,ENSG00000263972,MIR1587
hsa-mir-3149,ENSG00000266712,MIR3149
hsa-mir-4259,ENSG00000266458,MIR4259


# Save the dataset

In [31]:
cur_data.save_curated_data_h5ad()

/Users/zakirov/Documents/GitHub/PerturbationCatalogue/data_exploration/curation_tools/curation_tools.py:327: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adata.obs = adata.obs.fillna(value=np.nan)
... storing 'dataset_id' as categorical
... storing 'data_modality' as categorical
... storing 'significance_criteria' as categorical
... storing 'perturbation_name' as categorical
... storing 'perturbed_target_coord' as categorical
... storing 'perturbed_target_chromosome' as categorical
... storing 'perturbed_target_ensg' as categorical
... storing 'perturbed_target_symbol' as categorical
... storing 'perturbed_target_biotype' as categorical
... storing 'guide_sequence' as categorical
... storing 'perturbation_type_label' as categorical
... storing 'perturbation_type_id' as

✅ Curated h5ad data saved to ../curated/h5ad/datlinger_2017_curated.h5ad


In [32]:
cur_data.save_curated_data_parquet(split_metadata=True, save_metadata_only=True)

✅ Metadata saved to ../curated/parquet/datlinger_2017_curated_metadata.parquet


# Upload to BigQuery


In [33]:
upload_parquet_to_bq(
    parquet_path='../curated/parquet/datlinger_2017_curated_metadata.parquet',
    bq_dataset_id='prj-ext-dev-pertcat-437314.perturb_seq',
    bq_table_name='metadata',
    key_columns=['dataset_id', 'sample_id'],
    verbose=True
)

2025-12-12 11:21:30,963 DEBUG google.auth._default: Checking None for explicit credentials as part of auth process...
2025-12-12 11:21:30,965 DEBUG google.auth._default: Checking Cloud SDK credentials as part of auth process...
2025-12-12 11:21:32,068 DEBUG google.auth._default: Checking None for explicit credentials as part of auth process...
2025-12-12 11:21:32,069 DEBUG google.auth._default: Checking Cloud SDK credentials as part of auth process...
2025-12-12 11:21:32,585 DEBUG google.cloud.bigquery.opentelemetry_tracing: This service is instrumented using OpenTelemetry. OpenTelemetry or one of its components could not be imported; please add compatible versions of opentelemetry-api and opentelemetry-instrumentation packages in order to get BigQuery Tracing data.
2025-12-12 11:21:32,587 DEBUG urllib3.util.retry: Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
2025-12-12 11:21:32,588 DEBUG google.auth.transport.requests: Making reques

Staging table: loading `.parquet` file ../curated/parquet/datlinger_2017_curated_metadata.parquet to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging...


2025-12-12 11:21:33,734 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "POST /upload/bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs?uploadType=resumable HTTP/1.1" 200 0
2025-12-12 11:21:34,207 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "PUT /upload/bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs?uploadType=resumable&upload_id=AHVrFxNLWzJE_fTI7VIjJhwA4hEHWu59_zo_hnPGFTYHjKzB-0M2mypaV16DEXS1rGvkQyjSn33E1NHh2oJV4MRmLbygUqu02t6TAcB-z8PHfrg HTTP/1.1" 200 14002
2025-12-12 11:21:34,327 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs/7c3cd0e0-6a2c-4c05-9bbb-d7185a891369?projection=full&location=europe-west2&prettyPrint=false HTTP/1.1" 200 None
2025-12-12 11:21:34,328 DEBUG google.api_core.retry: Retrying due to , sleeping 0.6s ...
2025-12-12 11:21:35,044 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/prj-ext-dev-pertcat-

Staging table: loaded 5905 rows to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging


2025-12-12 11:21:36,813 DEBUG google.auth._default: Checking None for explicit credentials as part of auth process...
2025-12-12 11:21:36,814 DEBUG google.auth._default: Checking Cloud SDK credentials as part of auth process...
2025-12-12 11:21:37,196 DEBUG urllib3.util.retry: Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
2025-12-12 11:21:37,198 DEBUG google.auth.transport.requests: Making request: POST https://oauth2.googleapis.com/token
2025-12-12 11:21:37,201 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): oauth2.googleapis.com:443
2025-12-12 11:21:37,319 DEBUG urllib3.connectionpool: https://oauth2.googleapis.com:443 "POST /token HTTP/1.1" 200 None
2025-12-12 11:21:37,321 DEBUG urllib3.connectionpool: Starting new HTTPS connection (1): bigquery.googleapis.com:443
2025-12-12 11:21:37,771 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs?pre

Staging table: added ingested_at timestamp column to prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging


2025-12-12 11:21:41,894 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/prj-ext-dev-pertcat-437314/datasets/perturb_seq/tables/metadata?prettyPrint=false HTTP/1.1" 200 None
2025-12-12 11:21:42,502 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "POST /bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs?prettyPrint=false HTTP/1.1" 200 None
2025-12-12 11:21:42,559 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/prj-ext-dev-pertcat-437314/jobs/0d36c5af-19e4-446e-a716-bbeb9f7c2ddc?projection=full&location=europe-west2&prettyPrint=false HTTP/1.1" 200 None
2025-12-12 11:21:45,262 DEBUG urllib3.connectionpool: https://bigquery.googleapis.com:443 "GET /bigquery/v2/projects/prj-ext-dev-pertcat-437314/queries/0d36c5af-19e4-446e-a716-bbeb9f7c2ddc?maxResults=0&location=europe-west2&prettyPrint=false HTTP/1.1" 200 None
2025-12-12 11:21:45,332 DEBUG urllib3.connectionpool: https://bigquery.go

Merge completed: staging → prj-ext-dev-pertcat-437314.perturb_seq.metadata with type-safe casting.
Staging table: deleted prj-ext-dev-pertcat-437314.perturb_seq.metadata_staging


# Upload to GC Storage


In [34]:
!gcloud storage cp ../curated/h5ad/datlinger_2017_curated.h5ad gs://perturbation-catalogue-lake/perturbseq/curated/

Copying file://../curated/h5ad/datlinger_2017_curated.h5ad to gs://perturbation-catalogue-lake/perturbseq/curated/datlinger_2017_curated.h5ad
  Completed files 1/1 | 131.4MiB/131.4MiB | 1.8MiB/s                           

Average throughput: 2.1MiB/s
